Tic Tac Toe
---

In [1]:
import numpy as np
import pickle
import time

### Board State
---
Reflect & Judge the state

2 players p1 and p2; p1 uses symbol 1 and p2 uses symbol 2, vacancy as 0

In [2]:
BOARD_ROWS = 3
BOARD_COLS = 3

class State:
    def __init__(self, p1, p2):
        self.board = np.zeros((BOARD_ROWS, BOARD_COLS))
        self.p1 = p1
        self.p2 = p2
        self.isEnd = False
        self.boardHash = None
        # init p1 plays first
        self.playerSymbol = 1
    
    # get unique hash of current board state
    def getHash(self):
        self.boardHash = str(self.board.reshape(BOARD_COLS*BOARD_ROWS))
        return self.boardHash
    
    def winner(self):
        # row
        for i in range(BOARD_ROWS):
            if sum(self.board[i, :]) == 3:
                self.isEnd = True
                return 1
            if sum(self.board[i, :]) == -3:
                self.isEnd = True
                return -1
        # col
        for i in range(BOARD_COLS):
            if sum(self.board[:, i]) == 3:
                self.isEnd = True
                return 1
            if sum(self.board[:, i]) == -3:
                self.isEnd = True
                return -1
        # diagonal
        diag_sum1 = sum([self.board[i, i] for i in range(BOARD_COLS)])
        if diag_sum1 == 3:
            self.isEnd = True
            return 1
        if diag_sum1 == -3:
            self.isEnd = True
            return -1
        
        diag_sum2 = sum([self.board[i, BOARD_COLS-i-1] for i in range(BOARD_COLS)])
        if diag_sum2 == 3:
            self.isEnd = True
            return 1
        if diag_sum2 == -3:
            self.isEnd = True
            return -1
        
        # tie
        # no available positions
        if len(self.availablePositions()) == 0:
            self.isEnd = True
            return 0
        # not end
        self.isEnd = False
        return None
    
    def availablePositions(self):
        positions = []
        for i in range(BOARD_ROWS):
            for j in range(BOARD_COLS):
                if self.board[i, j] == 0:
                    positions.append((i, j))  # need to be tuple
        return positions
    
    def updateState(self, position):
        self.board[position] = self.playerSymbol
        # switch to another player
        self.playerSymbol = -1 if self.playerSymbol == 1 else 1
    
    # only when game ends
    def giveReward(self, result):
        if result == 1:
            self.p1.feedReward(1)
            self.p2.feedReward(0)
        elif result == -1:
            self.p1.feedReward(0)
            self.p2.feedReward(1)
        else:
            self.p1.feedReward(0.5)
            self.p2.feedReward(0.5)
    
    # board reset
    def reset(self):
        self.board = np.zeros((BOARD_ROWS, BOARD_COLS))
        self.boardHash = None
        self.isEnd = False
        self.playerSymbol = 1
    
    def play(self, rounds=1, showTraining=False, showGame=True, showDelay=0):
        p1_win_count = 0
        p2_win_count = 0
        draw_count = 0
        for i in range(rounds):
            if (i == 0 or (i + 1) % 1000 == 0 or (i + 1 == rounds)):
                if (showTraining):
                    print("Round: %d. P1 win count: %d. Draw count: %d. P2 win count: %d. P1 exp_rate: %f. P2 exp_rate: %f" % (
                        i + 1,
                        p1_win_count,
                        draw_count,
                        p2_win_count,
                        p1.explore_rate if hasattr(p1, "explore_rate") else 0,
                        p2.explore_rate if hasattr(p2, "explore_rate") else 0
                    ))
                p1_win_count = 0
                p2_win_count = 0
                draw_count = 0
            
            if (showGame):
                print("\n******************** Round %d ********************" % (i + 1))
            
            player = 0
            while not self.isEnd:
                if (showGame):
                    self.showBoard()
                    time.sleep(showDelay)
                    
                win = self.winner()
                if win is not None:
                    # self.showBoard()
                    # ended with p2 either win or draw
                    self.giveReward(win)
                    self.p1.reset()
                    self.p2.reset()
                    self.reset()

                    if win == 1:
                        p1_win_count += 1
                        if (showGame):
                            print(self.p1.name, "wins!")
                    elif win == -1:
                        p2_win_count += 1
                        if (showGame):
                            print(self.p2.name, "wins!")
                    else:
                        draw_count += 1
                        if (showGame):
                            print("Tie!")

                    break
                    
                if (player == 0):
                    # Player 1
                    positions = self.availablePositions()
                    p1_action = self.p1.chooseAction(positions, self.board, self.playerSymbol)
                    # take action and upate board state
                    self.updateState(p1_action)
                    board_hash = self.getHash()
                    self.p1.addState(board_hash)
                    # check board status if it is end
                else:
                    # Player 2
                    positions = self.availablePositions()
                    p2_action = self.p2.chooseAction(positions, self.board, self.playerSymbol)
                    self.updateState(p2_action)
                    board_hash = self.getHash()
                    self.p2.addState(board_hash)
                    
                player ^= 1 # switch player
    
    def showBoard(self):
        # p1: x  p2: o
        for i in range(0, BOARD_ROWS):
            print('-------------')
            out = '| '
            for j in range(0, BOARD_COLS):
                if self.board[i, j] == 1:
                    token = 'x'
                if self.board[i, j] == -1:
                    token = 'o'
                if self.board[i, j] == 0:
                    token = ' '
                out += token + ' | '
            print(out)
        print('-------------')    

In [3]:
class Player:
    def __init__(self, name, learning_rate=0.2, reward_decay=0.9, explore_rate=1, explore_rate_decay=0.9999, humanCheck=False):
        self.states = []  # record all positions taken
        self.states_value = {}  # Q-table. state -> value
        
        self.name = name
        self.learning_rate = learning_rate
        self.reward_decay = reward_decay
        self.explore_rate = explore_rate
        self.explore_rate_decay = explore_rate_decay
        self.humanCheck=humanCheck
        
    def getHash(self, board):
        boardHash = str(board.reshape(BOARD_COLS*BOARD_ROWS))
        return boardHash
    
    def chooseAction(self, positions, current_board, symbol):
        if np.random.uniform(0, 1) <= self.explore_rate:
            # take random action
            idx = np.random.choice(len(positions))
            action = positions[idx]
        else:
            values = np.zeros((3, 3))
            value_max = -999
            for p in positions:
                next_board = current_board.copy()
                next_board[p] = symbol
                next_boardHash = self.getHash(next_board)
                value = 0 if self.states_value.get(next_boardHash) is None else self.states_value.get(next_boardHash)
                values[p] = value
                if value >= value_max:
                    value_max = value
                    action = p
                    
            if (self.humanCheck):
                print(values)
                print("Recommended action: %s" % str(action))
                while True:
                    row = int(input("Input your action row:"))
                    col = int(input("Input your action col:"))
                    action = (row, col)
                    if action in positions:
                        break
                
        # print("{} takes action {}".format(self.name, action))
        self.explore_rate *= self.explore_rate_decay
        return action
    
    # append a hash state
    def addState(self, state):
        self.states.append(state)
    
    # at the end of game, backpropagate and update states value
    def feedReward(self, reward):
        for st in reversed(self.states):
            if self.states_value.get(st) is None:
                self.states_value[st] = 0
            self.states_value[st] += self.learning_rate*(reward - self.states_value[st])
            reward = self.states_value[st] * self.reward_decay
            
    def reset(self):
        self.states = []
        
    def savePolicy(self):
        fw = open('policy_' + str(self.name) + '.save', 'wb')
        pickle.dump(self.states_value, fw)
        fw.close()

    def loadPolicy(self, file):
        fr = open(file,'rb')
        self.states_value = pickle.load(fr)
        fr.close()

In [4]:
class HumanPlayer:
    def __init__(self, name):
        self.name = name 
    
    def chooseAction(self, positions, current_board, symbol):
        while True:
            row = int(input("Input your action row:"))
            col = int(input("Input your action col:"))
            action = (row, col)
            if action in positions:
                return action
    
    # append a hash state
    def addState(self, state):
        pass
    
    # at the end of game, backpropagate and update states value
    def feedReward(self, reward):
        pass
            
    def reset(self):
        pass

### Training

In [5]:
p1 = Player("first", learning_rate=0.2, reward_decay=0.9, explore_rate=1, explore_rate_decay=0.99999)
p2 = Player("second", learning_rate=0.2, reward_decay=0.9, explore_rate=1, explore_rate_decay=0.99999)

st = State(p1, p2)
print("Start training...")
st.play(rounds = 10000, showTraining = True, showGame = False, showDelay=1)
print("Finished!")

Start training...
Round: 1. P1 win count: 0. Draw count: 0. P2 win count: 0. P1 exp_rate: 1.000000. P2 exp_rate: 1.000000
Round: 1000. P1 win count: 589. Draw count: 130. P2 win count: 280. P1 exp_rate: 0.959033. P2 exp_rate: 0.965953
Round: 2000. P1 win count: 587. Draw count: 137. P2 win count: 276. P1 exp_rate: 0.919955. P2 exp_rate: 0.933326
Round: 3000. P1 win count: 568. Draw count: 141. P2 win count: 291. P1 exp_rate: 0.883326. P2 exp_rate: 0.902542
Round: 4000. P1 win count: 603. Draw count: 115. P2 win count: 282. P1 exp_rate: 0.848410. P2 exp_rate: 0.873113
Round: 5000. P1 win count: 575. Draw count: 125. P2 win count: 300. P1 exp_rate: 0.815087. P2 exp_rate: 0.844711
Round: 6000. P1 win count: 617. Draw count: 113. P2 win count: 270. P1 exp_rate: 0.783549. P2 exp_rate: 0.817977
Round: 7000. P1 win count: 605. Draw count: 114. P2 win count: 281. P1 exp_rate: 0.753368. P2 exp_rate: 0.792145
Round: 8000. P1 win count: 605. Draw count: 126. P2 win count: 269. P1 exp_rate: 0.7244

In [6]:
p1 = Player("first", learning_rate=0.2, reward_decay=0.9, explore_rate=0.1, explore_rate_decay=1)
p1.loadPolicy("policy_first.save")

p2 = Player("second", learning_rate=0.2, reward_decay=0.9, explore_rate=0.1, explore_rate_decay=1)
p2.loadPolicy("policy_second.save")

st = State(p1, p2)
print("Start training...")
st.play(rounds = 10000, showTraining = True, showGame = False, showDelay=1)
print("Finished!")

Start training...
Round: 1. P1 win count: 0. Draw count: 0. P2 win count: 0. P1 exp_rate: 0.100000. P2 exp_rate: 0.100000
Round: 1000. P1 win count: 189. Draw count: 765. P2 win count: 45. P1 exp_rate: 0.100000. P2 exp_rate: 0.100000
Round: 2000. P1 win count: 174. Draw count: 749. P2 win count: 77. P1 exp_rate: 0.100000. P2 exp_rate: 0.100000
Round: 3000. P1 win count: 205. Draw count: 712. P2 win count: 83. P1 exp_rate: 0.100000. P2 exp_rate: 0.100000
Round: 4000. P1 win count: 195. Draw count: 741. P2 win count: 64. P1 exp_rate: 0.100000. P2 exp_rate: 0.100000
Round: 5000. P1 win count: 205. Draw count: 710. P2 win count: 85. P1 exp_rate: 0.100000. P2 exp_rate: 0.100000
Round: 6000. P1 win count: 198. Draw count: 760. P2 win count: 42. P1 exp_rate: 0.100000. P2 exp_rate: 0.100000
Round: 7000. P1 win count: 187. Draw count: 737. P2 win count: 76. P1 exp_rate: 0.100000. P2 exp_rate: 0.100000
Round: 8000. P1 win count: 220. Draw count: 721. P2 win count: 59. P1 exp_rate: 0.100000. P2 e

In [7]:
p1.savePolicy()
p2.savePolicy()

### Computer vs Computer

In [8]:
p1 = Player("x", explore_rate=0, humanCheck=False)
p1.loadPolicy("policy_first.save")
# print(p1.states_value)

p2 = Player("o", explore_rate=0, humanCheck=True)
p2.loadPolicy("policy_second.save")

st = State(p1, p2)
st.play(rounds = 1, showTraining = False, showGame = True, showDelay=1)


******************** Round 1 ********************
-------------
|   |   |   | 
-------------
|   |   |   | 
-------------
|   |   |   | 
-------------
-------------
|   |   |   | 
-------------
|   | x |   | 
-------------
|   |   |   | 
-------------
[[0.25217127 0.0392635  0.17108114]
 [0.00645224 0.         0.00571563]
 [0.27656718 0.06009131 0.23448526]]
Recommended action: (2, 0)
Input your action row:0
Input your action col:0
-------------
| o |   |   | 
-------------
|   | x |   | 
-------------
|   |   |   | 
-------------
-------------
| o |   | x | 
-------------
|   | x |   | 
-------------
|   |   |   | 
-------------
[[0.00000000e+00 1.89436310e-04 0.00000000e+00]
 [1.12504366e-02 0.00000000e+00 7.43655987e-04]
 [2.94856787e-01 6.55119937e-04 6.99188519e-03]]
Recommended action: (2, 0)
Input your action row:2
Input your action col:0
-------------
| o |   | x | 
-------------
|   | x |   | 
-------------
| o |   |   | 
-------------
-------------
| o |   | x | 
-----------

### Human vs Computer

In [ ]:
p1 = Player("computer", explore_rate=0)
p1.loadPolicy("policy_first.save")

p2 = HumanPlayer("human")

st = State(p1, p2)
st.play(rounds = 1, showTraining = False, showGame = True, showDelay=1)


******************** Round 1 ********************
-------------
|   |   |   | 
-------------
|   |   |   | 
-------------
|   |   |   | 
-------------
-------------
|   |   |   | 
-------------
|   | x |   | 
-------------
|   |   |   | 
-------------


### Computer vs Human

In [10]:
p1 = HumanPlayer("human")

p2 = Player("computer", explore_rate=0)
p2.loadPolicy("policy_second.save")
2
st = State(p1, p2)
st.play(rounds = 1, showTraining = False, showGame = True, showDelay=1)